In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 1. 페이지 소스를 수집
q = '엑셀'
page = 1
url = f'http://www.yes24.com/Product/Search?domain=BOOK&query={q}&page={page}'
req = requests.get(url)

In [3]:
# 2. HTML태그 찾기
soup = BeautifulSoup(req.content, 'html.parser')
soup


<!DOCTYPE html >

<html lang="ko">
<head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
<meta content="86400" http-equiv="Accept-CH-Lifetime"/>
<meta content="unsafe-url" name="referrer"/>
<meta content="width=1170" name="viewport"/>
<title> 엑셀 - YES24</title>
<meta content="YES24" name="title"/>
<meta content="YES24는 대한민국 1위 인터넷 온라인서점입니다. 국내 최대의 도서정보를 보유하고 있으며 음반, DVD, 공연까지 다양한 문화콘텐츠 및 서비스를 제공합니다." name="description"/>
<meta content="인터넷 서점, 온라인 쇼핑, 상품 추천, 쇼핑몰, 상품 검색, 도서 정보, 국내도서, 외국도서, 전자책, eBook, 이북, 크레마, 공연, 콘서트, 뮤지컬, 음반, 예매, DVD, 블루레이, 예스24, YES24, 교보문고, 알라딘" name="keywords"/>
<link href="//www.yes24.com/OpensearchDescription.xml" rel="search" title="YES24" type="application/opensearchdescription+xml"/>
<script src="https://image.yes24.com/sysimage/Conten

In [4]:
soup_tag = soup.find('ul', attrs={'id' : 'yesSchList'})
soup_tags = soup_tag.find_all('li')
len(soup_tags)

24

In [5]:
temp = soup_tags[-1]
temp

<li>
<div class="itemUnit">
<div class="item_img">
<div class="img_canvas">
<span class="img_item">
<span class="img_grp">
<a class="lnk_img" href="/Product/Goods/86006680" onclick="setSCode('101_005_003_001'); setGoodsClickExtraCodeHub('029', '엑셀', '86006680', '0')"><em class="img_bdr"><img alt="엑셀 기초" border="0" class="lazy" data-original="https://image.yes24.com/goods/86006680/L" src="https://image.yes24.com/sysimage/renew/loadSpace.png"/></em></a>
</span>
</span>
</div>
<div class="img_btn">
<a class="btnC btn_preview" href="javascript:yes24GU.openPreviewCheck(86006680); setSCode('101_005_003_009');"><span class="bWrap"><em class="txt">미리보기</em></span></a>
</div>
</div>
<div class="item_info">
<div class="info_row info_keynote">
<span class="gd_keynote" id="spanGdKeynote">
<a class="lnk_series" href="javascript:void(0);" onclick="openUrl('/24/Category/Series/001?SeriesNumber=266540','Pcode','003_001')">능력향상 시즌3</a>
</span>
<script type="text/javascript">
    if ($('#spanGdKeynote')

In [6]:
# 3. 하나의 태그 수집
li_tag = soup_tags[0]

title = li_tag.find('a', attrs={'class' : 'gd_name'}).text
author = li_tag.find('span', attrs={'class' : 'authPub info_auth'}).text
print(title, author)

진짜 쓰는 실무 엑셀 
전진권 저
                                        


In [7]:
import re
author = re.sub('(\n|\r)', '', author)
author = author.rstrip()
author

'전진권 저'

In [8]:
pub = li_tag.find('span', attrs={'class' : 'authPub info_pub'}).text

price = li_tag.find('div', attrs={'class' : 'info_row info_price'}).find(attrs={'class' : 'txt_num'}).text
price = price.replace('원', '').replace(',', '')
print(pub, price)

제이펍 18900


In [9]:
try:
    price = int(price)
except:
    price = 0
price

18900

In [10]:
sale_idx = li_tag.find('span', attrs={'class' : 'saleNum'}).text
sale_idx = re.sub('\D', '', sale_idx)
sale_idx = int(sale_idx)
sale_idx

218511

In [11]:
pd.DataFrame([{'title' : title, 'author' : author, 'pub' : pub, 'price' : price, 'sale_idx' : sale_idx}])

,title,author,pub,price,sale_idx
0,진짜 쓰는 실무 엑셀,전진권 저,제이펍,18900,218511


In [12]:
# 4. for문
tot = pd.DataFrame()

for li_tag in soup_tags:
    title = li_tag.find('a', attrs={'class' : 'gd_name'}).text
    author = li_tag.find('span', attrs={'class' : 'authPub info_auth'}).text

    import re
    author = re.sub('(\n|\r)', '', author)
    author = author.rstrip()

    pub = li_tag.find('span', attrs={'class' : 'authPub info_pub'}).text

    price = li_tag.find('div', attrs={'class' : 'info_row info_price'}).find(attrs={'class' : 'txt_num'}).text
    price = price.replace('원', '').replace(',', '')

    try:
        price = int(price)
    except:
        price = 0

    try:
        sale_idx = li_tag.find('span', attrs={'class' : 'saleNum'}).text
        sale_idx = re.sub('\D', '', sale_idx)
        sale_idx = int(sale_idx)
    except:
        sale_idx = 0

    temp = pd.DataFrame([{'title' : title, 'author' : author, 'pub' : pub, 'price' : price, 'sale_idx' : sale_idx}])
    tot = pd.concat([tot, temp], 0)

<ipython-input-12-6286c52db3ce>:30: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  tot = pd.concat([tot, temp], 0)


In [13]:
tot.head()

,title,author,pub,price,sale_idx
0,진짜 쓰는 실무 엑셀,전진권 저,제이펍,18900,218511
0,회사에서 바로 통하는 실무 엑셀+파워포인트+워드&한글,"전미진, 이화진, 신면철 저",한빛미디어,19800,30423
0,회사는 엑셀을 가르쳐주지 않아요,쏘피(박성희) 저,메가스터디북스,16200,21996
0,된다! 실무 엑셀 파워포인트 워드&한글,"한정희, 이충욱 저",이지스퍼블리싱,27000,8706
0,실무에 바로 쓰는 일잘러의 엑셀 데이터 분석,에이블런 저,제이펍,16650,7746


In [14]:
from tqdm import tqdm
import time
import random
def random_sleep(fr=5,to=10):
    time.sleep(random.randint(fr, to) / 10)

#4.  함수로 만듭니다.
def crawl_1p(q, page):
    url = f'http://www.yes24.com/Product/Search?domain=BOOK&query={q}&page={page}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    soup_tag = soup.find('ul', attrs={'id':'yesSchList'})
    soup_tags = soup_tag.find_all('li')
    len(soup_tag)

    tot = pd.DataFrame()

    for li_tag in soup_tags:

        title = li_tag.find('a', attrs={'class': 'gd_name'}).text
        author = li_tag.find('span', attrs={'class': 'authPub info_auth'}).text

        import re
        author = re.sub('(\n|\r)', '', author)
        author = author.rstrip()

        pub = li_tag.find('span', attrs={'class': 'authPub info_pub'}).text

        price = li_tag.find('div', attrs={'class': 'info_row info_price'}).find(attrs={'class': 'txt_num'}).text
        price = price.replace('원', '').replace(',', '')

        try:
            price = int(price)
        except:
            price = 0

        try:
            sale_idx = li_tag.find('span', attrs={'class': 'saleNum'}).text
            sale_idx = re.sub('\D', '', sale_idx)
            sale_idx = int(sale_idx)
        except:
            sale_idx = 0

        temp = pd.DataFrame([{'title': title, 'author': author, 'pub': pub, 'price': price, 'sale_idx': sale_idx}])
        tot = pd.concat([tot, temp], 0)

    return tot

def crawl(q,to_page):
    """
    여러 페이지를 크롤링한다.
    :param q:
    :param to_page:
    :return:
    """
    tot = pd.DataFrame()
    for i in tqdm(range(1,to_page+1)):
        temp = crawl_1p(q,to_page)
        if len(temp)==0:
            print(f'{i} page has no list!')
            return tot

        tot = pd.concat([temp,tot], 0)
        random_sleep()

    return tot

if __name__ == '__main__':
    tot = crawl('엑셀', 3)

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-14-8ce20c976c1e>:45: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  tot = pd.concat([tot, temp], 0)
<ipython-input-14-8ce20c976c1e>:63: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  tot = pd.concat([temp,tot], 0)
100%|██████████| 3/3 [00:08<00:00,  2.99s/it]


In [15]:
tot.head()

,title,author,pub,price,sale_idx
0,엑셀Ⅹ파이썬 코드 레시피 125,"카네히로 카즈미 저/김은철, 유세라 역",AK(에이케이 커뮤니케이션즈),23750,666
0,일 잘하는 직장인을 위한 엑셀 자동화 with 파이썬,최은석 저,위키북스,27000,1914
0,엑셀 2016 기초,장경숙 저,시대인,9000,2334
0,시나공 ITQ 엑셀,길벗알앤디 저,길벗,15300,942
0,ITQ 엑셀2016,서명원 저,REXmedia(렉스미디어),12600,1101


In [16]:
import plotly.express as px

px.bar(data_frame=tot, x='title', y='sale_idx').update_xaxes(categoryorder = 'total descending')